In [1]:
# test_more_modular_usage.ipynb

# import sys
# sys.path.append("..")  # Adjust to point to your package location

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# ---------------------
# Part 1: classical_monte_carlo
# ---------------------
#  We'll import the helper functions individually
from qre_library.classical_monte_carlo import (
    simulate_stock_paths,
    compute_monte_carlo_payoff,
    compute_confidence_interval,
    black_scholes_call_price,
    run_classical_monte_carlo
)

# ---------------------
# Part 2: quantum_qae
# ---------------------
#  We'll import the helper functions individually
from qre_library.quantum_qae import (
    compute_lognormal_parameters,
    build_log_normal_distribution,
    compute_black_scholes_call,
    run_qae_estimation,
    run_quantum_qae
)

# ---------------------
# Part 3: quantum_statevector
# ---------------------
#  We'll import the helper functions individually
from qre_library.quantum_statevector import (
    compute_lognormal_parameters as compute_lognormal_params_sv,
    build_lognormal_distribution,
    compute_black_scholes_call as compute_bs_call_sv,
    compute_discrete_payoff,
    compute_statevector_payoff,
    run_quantum_statevector
)

# Common parameters
S0 = 50
K = 55
r = 0.05
sigma = 0.4
T = 30/365
t = 30
M = 10000
seed = 42

# --- 1. Using modular functions (Classical MC) step by step ---
print("=== Classical Monte Carlo (Step-by-Step) ===")
# A) Simulate paths
S_paths = simulate_stock_paths(S0=S0, r=r, sigma=sigma, T=T, t=t, M=M, seed=seed)

# B) Compute MC payoff
mc_price, payoffs = compute_monte_carlo_payoff(S_paths, K, r, T)

# C) Confidence interval
conf_int = compute_confidence_interval(payoffs, mc_price, r, T)

# D) Black-Scholes
bs_price_mc = black_scholes_call_price(S0, K, r, sigma, T)

# E) Compare
error_mc = abs(mc_price - bs_price_mc)

print(f"Step-by-Step MC Price:      {mc_price:.4f}")
print(f"Step-by-Step MC 95% CI:     {conf_int}")
print(f"Black–Scholes:              {bs_price_mc:.4f}")
print(f"Absolute Error vs. BS:      {error_mc:.4f}\n")

# F) (Optional) Compare with the all-in-one function
results_mc = run_classical_monte_carlo(S0, K, r, sigma, T, t, M, seed)
print(f"run_classical_monte_carlo result: {results_mc['mc_estimate']:.4f} (should match step-by-step)\n")

# (Optional) Plot histogram of final prices
# plt.hist(S_paths[-1, :], bins=50, alpha=0.7)
# plt.title("Distribution of Simulated Final Prices")
# plt.xlabel("Price")
# plt.ylabel("Frequency")
# plt.show()


# --- 2. Using modular functions (Quantum QAE) step by step ---
print("=== Quantum QAE (Step-by-Step) ===")
# A) Compute lognormal parameters
(mu, sigma_sq, mean, variance, stddev, low, high) = compute_lognormal_parameters(S0, r, sigma, T)

# B) Build lognormal distribution
uncertainty_model = build_log_normal_distribution(
    num_qubits=5,  # example
    mu=mu,
    sigma_sq=sigma_sq,
    low=low,
    high=high
)

# C) Build Qiskit's EuropeanCallPricing object
from qiskit_finance.applications.estimation import EuropeanCallPricing
if (high - K) != 0:
    rescaling_factor = 1.0 / (high - K)
else:
    rescaling_factor = 1.0

european_call_pricing = EuropeanCallPricing(
    num_state_qubits=5,   # must match the number of qubits
    strike_price=K,
    rescaling_factor=rescaling_factor,
    bounds=(low, high),
    uncertainty_model=uncertainty_model
)

# D) Black–Scholes for reference
bs_price_qae = compute_black_scholes_call(S0, K, r, sigma, T)

# E) Run QAE estimation
qae_result = run_qae_estimation(
    european_call_pricing=european_call_pricing,
    shots=10000,
    seed=seed
)

# F) Interpret QAE result
estimated_payoff = european_call_pricing.interpret(qae_result)
discounted_estimated_value = np.exp(-r*T) * estimated_payoff

# G) Confidence interval (denormalize)
conf_int_qae = np.array(qae_result.confidence_interval_processed)
discounted_conf_int_qae = [np.exp(-r * T) * c for c in conf_int_qae]
error_qae = abs(bs_price_qae - discounted_estimated_value)

print(f"Step-by-Step QAE Payoff (discounted): {discounted_estimated_value:.4f}")
print(f"Step-by-Step QAE 95% CI:              {discounted_conf_int_qae}")
print(f"Black–Scholes:                        {bs_price_qae:.4f}")
print(f"Absolute Error vs. BS:                {error_qae:.4f}\n")

# H) Compare with the all-in-one function
results_qae = run_quantum_qae(S=S0, K=K, r=r, sigma=sigma, T=T,
                              num_uncertainty_qubits=5,
                              shots=10000, seed=seed)
print(f"run_quantum_qae result:               {results_qae['discounted_estimated_value']:.4f} (should match step-by-step)\n")


# --- 3. Using modular functions (Quantum Statevector) step by step ---
print("=== Quantum Statevector (Step-by-Step) ===")
# A) Compute lognormal parameters
(mu_sv, sigma_sq_sv, mean_sv, var_sv, stddev_sv, low_sv, high_sv) = compute_lognormal_params_sv(S0, r, sigma, T)

# B) Build distribution
uncertainty_model_sv = build_lognormal_distribution(
    num_qubits=10,
    mu=mu_sv,
    sigma_sq=sigma_sq_sv,
    low=low_sv,
    high=high_sv
)

# C) Compute discrete payoff
discrete_value = compute_discrete_payoff(uncertainty_model_sv, K)

# D) Compute Black-Scholes
bs_price_sv = compute_bs_call_sv(S0, K, r, sigma, T)

# E) Statevector payoff
statevector_payoff = compute_statevector_payoff(uncertainty_model_sv, K)

# F) Discount
discounted_discrete_value = discrete_value * np.exp(-r*T)
discounted_statevector_value = statevector_payoff * np.exp(-r*T)

# G) Error vs. BS
error_vs_bs = abs(bs_price_sv - discounted_statevector_value)

print(f"Step-by-Step Discrete payoff (undisc.):   {discrete_value:.4f}")
print(f"Step-by-Step Statevector payoff (undisc.):{statevector_payoff:.4f}")
print(f"Discounted Discrete Payoff:               {discounted_discrete_value:.4f}")
print(f"Discounted Statevector Payoff:            {discounted_statevector_value:.4f}")
print(f"Black–Scholes:                            {bs_price_sv:.4f}")
print(f"Error vs. Black–Scholes:                  {error_vs_bs:.4f}\n")

# H) Compare with the all-in-one function
results_sv = run_quantum_statevector(S=S0, K=K, r=r, sigma=sigma, T=T,
                                     num_uncertainty_qubits=10)
print(f"run_quantum_statevector result (discounted payoff): {results_sv['discounted_statevector_payoff']:.4f} (should match step-by-step)\n")

# -------------------------------------------------------------------------
# Summary
# -------------------------------------------------------------------------
print("=== Summary of All Step-by-Step Approaches ===")
print(f"Classical MC:      {mc_price:.4f} (error vs BS = {error_mc:.4f})")
print(f"Quantum QAE:       {discounted_estimated_value:.4f} (error vs BS = {error_qae:.4f})")
print(f"Quantum Statevec:  {discounted_statevector_value:.4f} (error vs BS = {error_vs_bs:.4f})")


=== Classical Monte Carlo (Step-by-Step) ===
Step-by-Step MC Price:      0.7335
Step-by-Step MC 95% CI:     (np.float64(0.6929243603389195), np.float64(0.774090006312337))
Black–Scholes:              0.7267
Absolute Error vs. BS:      0.0068

run_classical_monte_carlo result: 0.7335 (should match step-by-step)

=== Quantum QAE (Step-by-Step) ===
Step-by-Step QAE Payoff (discounted): 0.4928
Step-by-Step QAE 95% CI:              [np.float64(0.483824301737962), np.float64(0.5016788546978239)]
Black–Scholes:                        0.7267
Absolute Error vs. BS:                0.2340

run_quantum_qae result:               0.4928 (should match step-by-step)

=== Quantum Statevector (Step-by-Step) ===
Step-by-Step Discrete payoff (undisc.):   0.6711
Step-by-Step Statevector payoff (undisc.):0.6711
Discounted Discrete Payoff:               0.6684
Discounted Statevector Payoff:            0.6684
Black–Scholes:                            0.7267
Error vs. Black–Scholes:                  0.0583

ru